## This notebook uses the OPENAI API to create a company brochure. It uses , BeautiulSoup to scrape the data from the website and pass it to the OpenAI API as input to summarize it in a company brochure. The resulting output is displayed nicely using MarkDown

## Import the libraries

In [28]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

# Connecting to OpenAI

Load in the environment variables in your `.env` file and connect to OpenAI.

In [29]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
    
MODEL = 'gpt-4o-mini'
URL = 'https://fusion5.com'
openai = OpenAI()

## Scrape the website and capture the results in the class
##### Beautiful Soup is a library that makes it easy to scrape information from web pages. It sits atop an HTML or XML parser, providing Pythonic idioms for iterating, searching, and modifying the parse tree

In [30]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## ChatGPT is used decide which links are relevant and replace relative links such as "/about" with "https://company.com/about". Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  

In [31]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [32]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [33]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

## Put all the details together and pass it onto another prompt to prepare the brochure.

In [34]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [35]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [36]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [40]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [41]:
create_brochure("F5", URL)

Found links: {'links': [{'type': 'about page', 'url': 'https://fusion5.com/au/about-us/'}, {'type': 'careers page', 'url': 'https://fusion5.com/au/careers/'}, {'type': 'contact page', 'url': 'https://fusion5.com/au/contact-us/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/fusion5/'}, {'type': 'people page', 'url': 'https://fusion5.com/au/our-people/'}]}


# Fusion5 Company Brochure

Welcome to Fusion5, your dedicated tech partner for transforming your business landscape. We're committed to streamlining operations and enhancing performance through innovative applications, services, and robust support.

## About Us

At Fusion5, we are more than just a software provider; we are your trusted systems integration partner. Our extensive portfolio includes top-tier enterprise solutions in **Enterprise Resource Planning (ERP)**, **Customer Relationship Management (CRM)**, **People Management**, **Cloud Services**, **Data and AI**, and **Cybersecurity**. We work closely with industry leaders like Microsoft, Oracle, and AWS to deliver customized services that meet the unique demands of your business.

### Our Vision
We aim to help businesses evolve by transforming challenges into opportunities. With cutting-edge technology and a customer-centric approach, we empower organizations to optimize operations and achieve their goals.

## Our Solutions

- **Enterprise Resource Planning (ERP):** Optimize performance with world-class ERP solutions from NetSuite, Microsoft, and Oracle.
- **Customer Relationship Management (CRM):** Automate multi-channel digital interactions and enhance customer value.
- **People Management:** Foster a people-centric culture with tailored solutions for HR, payroll, and performance management.
- **Planning, Data & Analytics:** Utilize data-driven insights to streamline reporting and analytics, ensuring alignment with business goals.
- **Managed Services:** Focus on your core business while we manage your IT needs, ensuring industrial-level security.
- **Cloud Services:** Benefit from flexible IT solutions, including on-premise, hybrid, or cloud setups.
- **Consultancy & Advisory:** Crafting tailored roadmaps for technology investments to drive tangible business success.
- **Cybersecurity:** Implement comprehensive strategies to safeguard your business against evolving security threats.

## Culture and Environment

At Fusion5, we prioritize a collaborative and inclusive culture. We believe that diverse perspectives lead to innovative solutions and foster a positive work environment. Our team is dedicated to continuous learning and development, supporting each other to stay ahead in a rapidly changing technological landscape.

## Careers at Fusion5

We are always looking for passionate, talented individuals to join our team. Whether you're a tech enthusiast or a seasoned professional, Fusion5 offers a vibrant work atmosphere with opportunities for growth and learning. Our career paths are designed to help you reach your potential while contributing to impactful projects that matter.

### Join Us
Explore our open positions and find your place at Fusion5 where we are #makingpotentialreality.

## Our Customers

Fusion5 serves a wide array of industries including:
- Agribusiness
- Financial Services
- Manufacturing
- Professional Services
- Wholesale Distribution

We partner with various organizations big and small to tailor technological solutions that drive efficiency and success.

---

**Ready to transform your business?** Reach out to us today and let’s chat about how we can work together.  
**Contact:** [Fusion5 Contact Page](#)  
**Follow Us:** [Fusion5 on Social Media](#)

Thank you for considering Fusion5—your partner in business transformation!

## Same data but stream the output data

In [42]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [43]:
stream_brochure("F5", URL)

Found links: {'links': [{'type': 'about page', 'url': 'https://fusion5.com/au/about-us/'}, {'type': 'careers page', 'url': 'https://fusion5.com/au/careers/'}, {'type': 'contact page', 'url': 'https://fusion5.com/au/contact-us/'}, {'type': 'company page', 'url': 'https://fusion5.com/au/'}]}



# Fusion5 Brochure

## Transform Your Business with Fusion5

At Fusion5, we’re more than just a software provider; we’re your dedicated systems integration partner. Our mission is to help businesses streamline operations, optimize performance, and unlock their full potential through cutting-edge technology solutions.

---

## **Our Core Offerings**

### **Enterprise Resource Planning (ERP)**
Maximize operational efficiency with world-class ERP systems from industry giants such as **NetSuite**, **Microsoft**, and **Oracle**. Our expert team ensures that your unique business requirements are met with precision.

### **Customer Relationship Management (CRM)**
Enhance customer interactions and drive value with our top-tier CRM solutions, designed for automated multi-channel digital engagement.

### **People Management**
Build a people-centric culture with our comprehensive HR solutions like **Jemini**, **EmpowerHR**, and **Jade Star**. We also offer enterprise-level outsourced payroll services for hassle-free management.

### **Planning, Data & Analytics**
Transform your relationship with data! Our Planning, Data, and Analytics solutions help your business match up accurate data and streamline reporting processes.

### **Managed Services**
Focus on your core competencies while we manage your IT resources. Our Managed Services protect your data and optimize expenditure without increasing headcount.

### **Cloud Solutions**
Boost your IT service performance with tailored on-premise, hybrid, or cloud solutions designed for enhanced user experiences.

### **Consultancy & Advisory**
Take advantage of our decades of cross-industry experience. We craft tailored roadmaps that align your technology investments with your strategic vision, turning ambition into reality.

### **Cybersecurity**
Ensure your business remains secure with our comprehensive cybersecurity services. We provide a strategic, monitored approach to safeguard your business from evolving threats.

---

## **Our Customers**
Fusion5 serves a diverse range of industries including:
- Agribusiness
- Financial Services
- Manufacturing
- Professional Services
- Wholesale Distribution

We partner with various strategic allies such as **Microsoft**, **Oracle**, and **Amazon Web Services** to guarantee top-notch service delivery.

---

## **Company Culture**
At Fusion5, we foster a collaborative and innovative environment. We believe in empowering our team members to contribute to the company’s success while continuously evolving professionally. Our culture promotes a people-centric approach, ensuring that both our clients and employees thrive.

---

## **Join Us! Careers at Fusion5**
We are always on the lookout for talented individuals who are passionate about technology and customer success. If you’re ready to make a difference and advance your career in a dynamic environment, check out our **[Careers](#)** page to explore current job openings.

---

## **Contact Us**
If you’re ready to transform your business with Fusion5's comprehensive solutions, don’t hesitate! **[Contact Us](#)** today for a consultation or to find out more about our offerings. 

Together, let's make potential reality.

---

**Fusion5 - AU | Fusion5 - NZ**

